# Run all these cells below for importing necessary libraries

In [ ]:
!pip install imageio
!pip install imgaug
!pip install Pillow

In [20]:
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
%matplotlib inline
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from PIL import Image, ImageFilter, ImageEnhance

In [21]:
import os

In [22]:
def convert_to_yolo (bbs):
    yolo=[]
    h=bbs.height
    w=bbs.width
    for i in range(len (bbs)):
        width=bbs[i].x2-bbs[i].x1
        height=bbs[i].y2-bbs[i].y1
        x_center=bbs[i].center_x
        y_center=bbs[i].center_y
        yolo_1=[x_center/w, y_center/h, width/w, height/h]
        yolo.append(yolo_1)
    return yolo

In [23]:
def pad(image, by):
    image_border1=ia.pad(image, top=1, right=1, bottom=1, left=1, mode="constant", cval=255)
    image_border2=ia.pad(image_border1, top=by-1, right=by-1, bottom=by-1, left=by-1, mode="constant", cval=0)
    return image_border2
def draw_bbs(image, bbs, border):
    GREEN= [0,255,0]
    ORANGE=[255,140,0]
    RED=[255,0,0]
    image_border=pad(image, border)
    for bb in bbs.bounding_boxes:
        if bb.is_fully_within_image(image.shape):
            color=GREEN
        elif bb.is_partly_within_image(image.shape):
            color=ORANGE
        else:
            color=RED
        image_border=bb.shift(left=border, top=border).draw_on_image(image_border, size=2, color=color)
    return image_border

In [24]:
def blur_a_part(x_min, y_min, x_max, y_max, image_path):
    img=Image.open(image_path)
    cropped_image=img.crop((x_min, y_min, x_max, y_max))
    blurred_image=cropped_image.filter(ImageFilter.GaussianBlur(radius=2))
    img.paste(blurred_image, (x_min, y_min, x_max, y_max))
    ia.imshow(bbs.draw_on_image(img, size=2))
    print(convert_to_yolo(bbs))

In [25]:
def fade_image(image, p1, p2, flow_up=False):
    fade_range = list(range(int(height*p1), int(height*p2)))
    fade_range = fade_range[::-1] if flow_up else fade_range
    for y in fade_range:
        if flow_up:
            alpha = int((y - height*p1)/height/(p2-p1) * 255)
        else:
            alpha = 255-int((y - height*p1)/height/(p2-p1) * 255)
        for x in range(width):
            pixels[x, y] = pixels[x, y][:3] + (alpha,) 


# Put path1 and path2 as the images and labels folder (before augmentation)

In [8]:
path1= #path should be similar to ..\\images\\train\\
path2=#path should be similar to ..\\labels\\train\\

# Put path3 and path4 as the images and labels folder (after augmentation)

In [9]:
path3= #path should be similar to ..\\images\\train\\
path4=#path should be similar to ..\\labels\\train\\

# Use this function for renaming files.

#### start will be the starting number for the name of the file.

In [15]:
def renamer(start):
    c1=start
    c2=start
    for filename in os.listdir(path1):
        my_dest=str(c1) + ".jpg"
        my_source = path1+ filename
        my_dest =path1 + my_dest
        os.rename(my_source,my_dest)
        c1=c1+1
    for filename in os.listdir(path2):
        my_dest=str(c2) + '.txt'
        my_source = path2+ filename
        my_dest =path2 + my_dest
        os.rename(my_source,my_dest)
        c2=c2+1


# Just run this function

In [10]:
def operate(i):
    #global bbs
    image=imageio.imread(path1 + str(i) + ".jpg")
#inorder to check labels iterate through this
    f=open(path2 + str(i) + ".txt")
    l=f.readlines()
    for i in range(len(l)):
    c1=l[0].split()
    h=image.shape[0]
    w=image.shape[1]
    c1[1]=float(c1[1])
    c1[2]=float(c1[2])
    c1[3]=float(c1[3])
    c1[4]=float(c1[4])
    x_center1=int(c1[1]*w)
    y_center1=int(c1[2]*h)
    xmin1=w/2*(2*c1[1] - c1[3])
    xmax1=w/2*(2*c1[1]+c1[3])
    ymin1=h/2*(2*c1[2]-c1[4])
    ymax1=h/2*(2*c1[2]+c1[4])
    obj_class1=c1[0]
    if len(l)==2:
        c2=l[1].split()
        c2[1]=float(c2[1])
        c2[2]=float(c2[2])
        c2[3]=float(c2[3])
        c2[4]=float(c2[4])
        x_center2=int(c2[1]*w)
        y_center2=int(c2[2]*h)
        xmin2=w/2*(2*c2[1] - c2[3])
        xmax2=w/2*(2*c2[1]+c2[3])
        ymin2=h/2*(2*c2[2]-c2[4])
        ymax2=h/2*(2*c2[2]+c2[4])
        obj_class2=c2[0]
    if len(l)==1:
        bbs=BoundingBoxesOnImage ([BoundingBox(x1=xmin1, x2=xmax1, y1=ymin1, y2=ymax1)], shape=image.shape)
    elif len(l)>=2:
        bbs=BoundingBoxesOnImage ([BoundingBox(x1=xmin1, x2=xmax1, y1=ymin1, y2=ymax1), BoundingBox(x1=xmin2, x2=xmax2, y1=ymin2, y2=ymax2)], shape=image.shape)
#we have derived the bounding boxes coordinates in the correct format.
    move=iaa.Affine(translate_percent={"x":0.10}, scale=0.8)
    image_move, bbs_move=move(image=image, bounding_boxes=bbs)
    horizontal_flip=iaa.Fliplr(1)
    flipped_image=horizontal_flip.augment_image(image)
    image_flipped, bbs_flip=horizontal_flip(image=image, bounding_boxes=bbs)
    dropout=iaa.CoarseDropout()
    dropout_image=dropout.augment_image(image)
    image_dropout, bbs_dropout=dropout(image=image, bounding_boxes=bbs)
    crop=iaa.Crop(px=(100,64,0,0))
    crop_image=crop.augment_image(image)
    image_aug, bbs_crop=crop(image=image, bounding_boxes=bbs)
    rotate=iaa.Affine(rotate=(-50,30))
    image_rot, bbs_rot=rotate(image=image, bounding_boxes=bbs)
    image_after=draw_bbs(image_rot, bbs_rot.remove_out_of_image().clip_out_of_image(), 100)
    im=Image.open(path1 + str(i) + ".jpg")
    cropped_image=im.crop((x_center1, y_center1, int(xmax1), int(ymax1)))
    blurred_image=cropped_image.filter(ImageFilter.GaussianBlur(radius=2))
    im.paste(blurred_image, (x_center1, y_center1, int(xmax1), int(ymax1)))
    cropped_image=im.crop((0, 0, int(w/2), int(h/2)))
    blurred_image=cropped_image.filter(ImageFilter.GaussianBlur(radius=5))
    im.paste(blurred_image, (0, 0, int(w/2), int(h/2)))
    fog=iaa.Fog()
    snowflakes= iaa.Snowflakes(flake_size=(0.55, 0.55), speed=(0.01, 0.05))
    rain= iaa.Rain(speed=(0.1, 0.3))
    snowflakes_image=snowflakes.augment_image(image)
    fog_image=fog.augment_image(image)
    rain_image=rain.augment_image(image)
    imz = Image.open(path1 +str(i) + ".jpg")
    imz.putalpha(177)
    width, height = imz.size
    pixels = imz.load()
    fade_image(pixels, 0.2 , 0.5, flow_up=True)
    fade_image(pixels, 0.5, 1, flow_up=False)
    l_0=convert_to_yolo(bbs)
    l0=convert_to_yolo(bbs_move)
    l1=convert_to_yolo(bbs_flip)
    l2=convert_to_yolo(bbs_dropout)
    l3=convert_to_yolo(bbs_crop)
    l4=convert_to_yolo(bbs_rot)
    l5=convert_to_yolo(bbs)
    f1=open(path4 +str (i) + "-0.txt", 'w')
    f1.write(str(obj_class1) + " ")
    for x in range (4):
        f1.write(str(l_0[0][x]) + " ")
    if len(l)==2:
        f1.write("\n")
        f1.write (str(obj_class2) + " ")
        for x in range(4):
            f1.write(str(l_0[1][x]) + " ")
    f1=open(path4 +str (i) + "-1.txt", 'w')
    f1.write(str(obj_class1) + " ")
    for x in range (4):
        f1.write(str(l0[0][x]) + " ")
    if len(l)==2:
        f1.write("\n")
        f1.write (str(obj_class2) + " ")
        for x in range(4):
            f1.write(str(l0[1][x]) + " ")
    f1=open(path4 +str (i) + "-2.txt", 'w')
    f1.write(str(obj_class1) + " ")
    for x in range (4):
        f1.write(str(l1[0][x]) + " ")
    if len(l)==2:
        f1.write("\n")
        f1.write (str(obj_class2) + " ")
        for x in range(4):
            f1.write(str(l1[1][x]) + " ")
    f1=open(path4 +str (i) + "-3.txt", 'w')
    f1.write(str(obj_class1) + " ")
    for x in range (4):
        f1.write(str(l2[0][x]) + " ")
    if len(l)==2:
        f1.write("\n")
        f1.write (str(obj_class2) + " ")
        for x in range(4):
            f1.write(str(l2[1][x]) + " ")
    f1=open(path4 +str (i) + "-4.txt", 'w')
    f1.write(str(obj_class1) + " ")
    for x in range (4):
        f1.write(str(l3[0][x]) + " ")
    if len(l)==2:
        f1.write("\n")
        f1.write (str(obj_class2) + " ")
        for x in range(4):
            f1.write(str(l3[1][x]) + " ")
    f1=open(path4 +str (i) + "-5.txt", 'w')
    f1.write(str(obj_class1) + " ")
    for x in range (4):
        if l4[0][x]>1:
            l4[0][x]=1
        f1.write(str(l4[0][x]) + " ")
    if len(l)==2:
        f1.write("\n")
        f1.write (str(obj_class2) + " ")
        for x in range(4):
            if l4[1][x]>1:
                l4[1][x]=1
            f1.write(str(l4[1][x]) + " ")
    f1=open(path4 +str (i) + "-6.txt", 'w')
    f1.write(str(obj_class1) + " ")
    for x in range (4):
        f1.write(str(l_0[0][x]) + " ")
    if len(l)==2:
        f1.write("\n")
        f1.write (str(obj_class2) + " ")
        for x in range(4):
            f1.write(str(l_0[1][x]) + " ")
    f1=open(path4 +str (i) + "-7.txt", 'w')
    f1.write(str(obj_class1) + " ")
    for x in range (4):
        f1.write(str(l_0[0][x]) + " ")
    if len(l)==2:
        f1.write("\n")
        f1.write (str(obj_class2) + " ")
        for x in range(4):
            f1.write(str(l_0[1][x]) + " ")
    f1=open(path4 +str (i) + "-8.txt", 'w')
    f1.write(str(obj_class1) + " ")
    for x in range (4):
        f1.write(str(l_0[0][x]) + " ")
    if len(l)==2:
        f1.write("\n")
        f1.write (str(obj_class2) + " ")
        for x in range(4):
            f1.write(str(l_0[1][x]) + " ")
    f1=open(path4 +str (i) + "-9.txt", 'w')
    f1.write(str(obj_class1) + " ")
    for x in range (4):
        f1.write(str(l_0[0][x]) + " ")
    if len(l)==2:
        f1.write("\n")
        f1.write (str(obj_class2) + " ")
        for x in range(4):
            f1.write(str(l_0[1][x]) + " ")
    f1=open(path4 +str (i) + "-10.txt", 'w')
    f1.write(str(obj_class1) + " ")
    for x in range (4):
        f1.write(str(l_0[0][x]) + " ")
    if len(l)==2:
        f1.write("\n")
        f1.write (str(obj_class2) + " ")
        for x in range(4):
            f1.write(str(l_0[1][x]) + " ")
    imageio.imwrite(path3 + str (i) + "-0.jpg", image)
    imageio.imwrite(path3 + str (i) + "-1.jpg", image_move)
    imageio.imwrite(path3 + str (i) + "-2.jpg", flipped_image)
    imageio.imwrite(path3 + str (i) + "-3.jpg", image_dropout)
    imageio.imwrite(path3 + str (i) + "-4.jpg", crop_image)
    imageio.imwrite(path3 + str (i) + "-5.jpg", image_rot)
    imageio.imwrite(path3 + str (i) + "-6.jpg", im)
    imageio.imwrite(path3 + str (i) + "-7.jpg", snowflakes_image)
    imageio.imwrite(path3 + str (i) + "-8.jpg", fog_image)
    imageio.imwrite(path3 + str (i) + "-9.jpg", rain_image)
    imz.save(path3 + str (i) + "-10.png")

In [2]:
list1=[]

In [ ]:
#set start as starting file before augmentation
#set end as ending file
start=None
end=None

In [ ]:
for iterator in range (start,end):
    try:
        if os.path.exists(path1 + str(iterator) + ".jpg"):
            operate(iterator)
        else:
            iterator=iterator+1
    except:
        list1.append(iterator)
        continue

C:\Users\hrida\anaconda3\lib\site-packages\imgaug\imgaug.py:106: DeprecationWarning: Using imgaug.imgaug.pad is deprecated. Use imgaug.augmenters.size.pad instead.
  warn(msg, category=DeprecationWarning, stacklevel=stacklevel)
C:\Users\hrida\anaconda3\lib\site-packages\imgaug\augmentables\utils.py:349: DeprecationWarning: Got one of the arguments `top` (100), `right` (None), `bottom` (None), `left` (100) in a shift() call. These are deprecated. Use `x` and `y` instead.
  ia.warn_deprecated(


# END OF FILE

In [29]:
"""
f=open("C:\\Users\\hrida\\Desktop\\a.txt", 'r')
#l=f.readlines()
#f.truncate(0)
f=open("C:\\Users\\hrida\\Desktop\\a.txt", 'w')
for i in range(len(l)):
    #f=open(path4 + "385.5.txt" , 'w')
    string=l[i]
    string=string.replace(",", "")
    m=string.split()
    for i in range(len(m)):
        if float(m[i])>1 and float(m[i])<2:
            m[i]=1
        f.write(str(m[i]))
        f.write(" ")
    f.write("\n")
    """
image = np.zeros((100, 100))
    
    
    

In [48]:
m=bbs[0]

AttributeError: 'BoundingBoxesOnImage' object has no attribute 'apend'